# Initialize

## Imports

In [2]:
import sys; module_name = '..\src'
if module_name not in sys.path: sys.path.insert(0, module_name)

import pandas as pd
import numpy as np
from pathlib import Path
from pprint import pprint
from neo4j import GraphDatabase, Session

%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
%autoreload 2

## Configure

In [4]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [5]:
# TODO Authentication is disabled
uri = "bolt://127.0.0.1:7687"
user = "neo4j"
pw = None
pw = "Mon_1028"
#pw = "neo4j"

## Define

In [6]:
def build_merge_file_query(file_id:int, directory:str, file_name:str):
    qry = f"""MERGE (f:File {{fileId:'{file_id}', directory:'{directory}', name:'{file_name}'}})
        RETURN f"""
    return qry

def merge_file_node(trx:Session, file_id:int, directory:str, file_name:str):
    qry = f"""MERGE (f:File {{fileId:'{file_id}', directory:'{directory}', name:'{file_name}'}})
        RETURN f"""
    print(qry)
    result = trx.run(qry)
    return result.values()

In [7]:
def read_into_dataframe(session, qry):
    result = session.run(qry)
    df = result.to_df(expand=True)
    return df

def get_graph(session:Session, qry:str):
    result = session.run(qry)
    g = result.graph()
    return g

def delete_node(session:Session, id:int):
    qry = f"""MATCH (n)
        WHERE id(n)={id}
        DETACH DELETE n"""
    result = session.run(qry)
    return result

# Interact

## Write

In [ ]:
with GraphDatabase.driver(uri, auth=(user, pw)) as driver: 
    driver.verify_connectivity()
    
    with driver.session() as session:
        qry = build_merge_file_query(2, "directory", "file_name3")
        results = session.run(qry)
        #results = session.execute_write(merge_file_node, 1, "directory", "file_name")
        df = results.to_df(expand=True)
        v = results.values()

df

## Read into dataframe

In [14]:
qry = """MATCH (n:Person) RETURN n.name AS name"""

with GraphDatabase.driver(uri, auth=(user, pw)) as driver: 
    driver.verify_connectivity()
    
    with driver.session() as session:
        df = read_into_dataframe(session, qry)
    
    with driver.session() as session:
        graph = get_graph(session, qry)
    
df

,name
0,Michael Angerthal
1,Danielle Vasilakos
2,Christopher Palumbo
3,Henry Hardaway
4,David Hanley
5,Mark Moots
6,Nicole Stephenson
7,Lina Patel
8,Robyn Battles
9,David Pellerin


KeyError: 0

In [ ]:
with GraphDatabase.driver(uri, auth=(user, pw)) as driver: 
    with driver.session() as session:
        r = delete_node(session, 0)

In [ ]:
r.consume().query